In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from cebra import CEBRA
from PIL import Image
import cv2
import torch
import pickle
import os

In [4]:
filepath_behavior = "2020_12_4_MV1_run_behavior"
image_sequence = []
for iter, file in enumerate(os.listdir(filepath_behavior)):
     filename = os.fsdecode(file)   
     if filename.endswith(".tif" ) and filename.split('_')[0] == 'move':
         image_sequence.append(cv2.imreadmulti(filepath_behavior + '/' + filename)[1])
         if iter > 500: break
         continue
     else:
         continue   

In [5]:
def flatten_data(data):
    data_flat = np.squeeze(data[0])
    for x in data[1::]:
        data_flat = np.concatenate((data_flat, np.squeeze(x)))
    return data_flat

In [6]:
image_sequence_flat = flatten_data(image_sequence)

In [7]:
# from https://huggingface.co/facebook/dino-vits8
# testing transformer
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests

processor = ViTImageProcessor.from_pretrained('facebook/dino-vits8')
model = ViTModel.from_pretrained('facebook/dino-vits8')
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/home/murphy/anaconda3/envs/cebra-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vits8 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def get_features(model, image):
  return model(**processor(images=image, return_tensors="pt").to(device)).pooler_output.cpu().detach().numpy()

In [9]:
from PIL import Image

def np_to_PIL(image_array):
    return [Image.fromarray(np.uint8(image)).convert('RGB') for image in image_array]

def process_behav(beh_seq):
  return np.array([get_features(model, np_to_PIL(beh_frame) ) for beh_frame in beh_seq])

In [10]:
embeddings_flat = process_behav(image_sequence_flat)

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.41 GiB (GPU 0; 23.65 GiB total capacity; 12.95 GiB already allocated; 2.04 GiB free; 13.07 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF